In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyngrok

In [3]:
!pip install ultralytics

  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.3.83
    Uninstalling nvidia-cusolver-cu12-11.6.3.83:
      Successfully uninstalled nvidia-cusolver-cu12-11.6.3.83


In [8]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import os
import cv2
import numpy as np
from ultralytics import YOLO
import base64
from io import BytesIO
from PIL import Image
import os


# Ngrok setup
ngrok.set_auth_token("2uumg7kLy68a0nmkmlC5QzvOBkt_4RADoBHWDVVr5pmf6vNuT")
public_url = ngrok.connect(5000).public_url


app = Flask(__name__)


# Load the YOLO model
model = YOLO("/content/drive/MyDrive/Driver Buddy AI/runs/detect/train2/weights/best.pt")

# Class names
class_names = ['abs', 'airbag', 'dipped beam', 'engine', 'hand brake',
               'power steer', 'seatbelt', 'stability control', 'tire pressure', 'warning']

@app.route('/detect', methods=['POST'])
def detect_objects():
    # Check if image was uploaded
    if 'image' not in request.files:
        return jsonify({'error': 'No image uploaded'}), 400

    # Read the uploaded image
    file = request.files['image']
    if file.filename == '':
        return jsonify({'error': 'No image selected'}), 400

    try:
        # Read image file
        image_bytes = file.read()
        nparr = np.frombuffer(image_bytes, np.uint8)
        image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Run inference
        results = model(image)

        # Process detections
        detections = []
        annotated_image = image.copy()
        height, width, _ = image.shape

        for result in results:
            for box in result.boxes:
                class_id = int(box.cls)
                confidence = float(box.conf)
                x_center, y_center, w, h = box.xywhn[0].tolist()  # Normalized coordinates

                # Convert normalized coordinates to pixel coordinates
                x1 = int((x_center - w / 2) * width)
                y1 = int((y_center - h / 2) * height)
                x2 = int((x_center + w / 2) * width)
                y2 = int((y_center + h / 2) * height)

                # Store detection info
                detection = {
                    'predicted_class': class_names[class_id],
                    'confidence': confidence,
                    'bounding_box': {
                        'x_center': x_center,
                        'y_center': y_center,
                        'width': w,
                        'height': h
                    },
                    'pixel_coordinates': {
                        'x1': x1,
                        'y1': y1,
                        'x2': x2,
                        'y2': y2
                    }
                }
                detections.append(detection)

                # Draw bounding box and label on the image
                cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(annotated_image,
                            f"{class_names[class_id]}",
                            (x1-10, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.6, (0, 255, 0), 2)

        # Convert annotated image to base64
        _, buffer = cv2.imencode('.jpg', cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR))
        base64_image = base64.b64encode(buffer).decode('utf-8')

        # Prepare response
        response = {
            'detections': detections,
            'annotated_image': base64_image,
            'message': 'Detection successful'
        }

        return jsonify(response), 200

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Display the public URL for accessing the app
print(f"To access the app globally, click here: {public_url}")

if __name__ == '__main__':
    app.run(port= 5000)

To access the app globally, click here: https://ae9e-34-81-129-120.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit



0: 640x640 1 airbag, 1 dipped beam, 1 hand brake, 1 seatbelt, 295.0ms
Speed: 11.3ms preprocess, 295.0ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)


INFO:werkzeug:127.0.0.1 - - [27/Mar/2025 20:56:44] "POST /detect HTTP/1.1" 200 -
